# Molecular Dynamics 4 

Ok, now that we have multiple atoms, lets have them interact with each other.  
  
So lets restart all of our previous turtle things to get window set up.

In [1]:
import turtle
import random
import math

When using random numbers, it is important to start a seed to insure that you actually are getting random numbers.

In [2]:
random.seed()

Now lets get our back drop set up.

In [3]:
window = turtle.Screen()
window.title('Molecular Dynamics 4')
window.clear()

Lets see how big our window is, which will tell us where the boundaries are.

In [4]:
# We will want to store these numbers in a variable
height = window.window_height()
width = window.window_width()

Lets check and see what we have here.

In [5]:
print height
print width

810
960


Now we have our window, lets make a turtles to draw our atoms. We are going to make a list of atoms and then populate it.

In [6]:
# Number of atoms we want
num_atoms = 20

# This initializes an empty list
atoms = []

# Now use a loop to initialize a new atom
# and do it num_atoms times.
for i in range(num_atoms):
    atoms.append(turtle.Turtle())

We should now have two atoms stored in atom_list. We can check various aspects of this new list using print.

In [7]:
print len(atoms)

20


In [8]:
print atoms

[<turtle.Turtle object at 0x10439ce90>, <turtle.Turtle object at 0x10439cf10>, <turtle.Turtle object at 0x10439cf90>, <turtle.Turtle object at 0x1043b3090>, <turtle.Turtle object at 0x1043b3150>, <turtle.Turtle object at 0x1043b3210>, <turtle.Turtle object at 0x1043b32d0>, <turtle.Turtle object at 0x1043b3390>, <turtle.Turtle object at 0x1043b3450>, <turtle.Turtle object at 0x1043b3510>, <turtle.Turtle object at 0x1043b35d0>, <turtle.Turtle object at 0x1043b3690>, <turtle.Turtle object at 0x1043b3750>, <turtle.Turtle object at 0x1043b3810>, <turtle.Turtle object at 0x1043b38d0>, <turtle.Turtle object at 0x1043b3990>, <turtle.Turtle object at 0x1043b3a50>, <turtle.Turtle object at 0x1043b3b10>, <turtle.Turtle object at 0x1043b3bd0>, <turtle.Turtle object at 0x1043b3c90>]


Everything looks good. Now lets draw our atoms as spheres with a fixed radius and random color and place them at random locations. Lets make sure we don't put any atoms on top of another. To help this, a distance function is useful. So lets write one!

In [9]:
# Lets name our function
def distance(atom1, atom2):
    
    # Inputs are the atoms, which contain
    # (x,y) coordinate pairs.
    # Distance is sqrt((x1-x2)^2 + (y1-y2)^2)
    
    # Lets get the coordinates of atom1
    (x1,y1) = atom1.pos()
    
    # Lets get the coordinates of atom2
    (x2,y2) = atom2.pos()

    # Lets calculate the distance
    d = math.sqrt((x1-x2)**2 + (y1-y2)**2)
    
    # And lets return our distance
    return d

In [10]:
# Variables to hold things we want to be constant
atom_radius = 20

# Scaling factor here is so we don't get our 
# initial positions stuck on the edge.
scaling_factor = 0.8

# We need to loop over each atom.
for i in range(num_atoms):
   
    # Draw the atom in the proper shape
    atoms[i].shape('circle')
    atoms[i].shapesize(atom_radius/10.0)
    atoms[i].color((random.random(),random.random(),random.random()))
    atoms[i].penup()
    atoms[i].goto(random.uniform(-1,1)*width/2.0 * scaling_factor, random.uniform(-1,1)*height/2.0 * scaling_factor)
  
    # Lets make sure this atom isn't on top of another atom
    # Lets get the minimum distance to all other atoms
    d = 1000000.0
    for j in range(0,i):
        d = min(d, distance(atoms[i],atoms[j]))
        print d
    # Check if we're too close
    while d < 2.0*atom_radius:
        
        # Reset d
        d = 1000000.0
        
        # Let us know we're too close
        print "Atom too close. Repositioning."
        print "   (%d,%d) = %f" % (i, j, d)
        # Choose a new random spot and remeasure
        atoms[i].goto(random.uniform(-1,1)*width/2.0 * scaling_factor, 
                      random.uniform(-1,1)*height/2.0 * scaling_factor)
        for j in range(0,i):
            d = min(d, distance(atoms[i],atoms[j]))
            print "      (%d,%d) = %f" % (i,j,d)
        
    # Turtles can be very slow. This is a semi-fix to tell
    # turtles not to update the screen with every change,
    # but rather wait till a set of updates are done and 
    # then update the screen.
    atoms[i].tracer(0,0)
    turtle.update()
    

676.311407722
431.90179345
251.099548718
265.582935992
265.582935992
265.582935992
587.292684034
131.933923072
131.933923072
131.933923072
330.125490702
330.125490702
164.540116257
164.540116257
164.540116257
348.856949662
348.856949662
198.843340981
198.843340981
198.843340981
198.843340981
357.559342981
357.559342981
357.559342981
107.771941545
107.771941545
107.771941545
107.771941545
583.520439445
123.692327454
123.692327454
123.692327454
14.758871589
14.758871589
14.758871589
14.758871589
Atom too close. Repositioning.
   (8,7) = 1000000.000000
      (8,0) = 174.064482
      (8,1) = 174.064482
      (8,2) = 174.064482
      (8,3) = 174.064482
      (8,4) = 174.064482
      (8,5) = 174.064482
      (8,6) = 174.064482
      (8,7) = 174.064482
533.588441473
452.333927057
403.31795803
324.293689972
322.504609634
322.504609634
242.861504523
221.070991269
221.070991269
582.39017817
307.475906642
307.475906642
307.475906642
184.212033348
184.212033348
184.212033348
184.212033348
184.2120

Alright, now lets make a vector with random velocities for each atom. Our velocity vector will have one line per atom, and each line will have a pair of floating point numbers that represent (velocity_x, velocity_y).

In [11]:
# Max velocity we want
max_velocity = 20.0

# Initailize an empty list
vel = []

# Now loop over the number of atoms
for i in range(num_atoms):
    vel.append([random.uniform(-1,1)*max_velocity, random.uniform(-1,1)*max_velocity])

Lets check to make sure this worked.

In [12]:
print len(vel)

20


In [13]:
print vel

[[4.655349207021269, -5.407684326983735], [-4.2539221389894655, 11.128647036278947], [12.069363246829766, 18.514877531833655], [-18.095272572416363, 13.709159376201079], [1.3420271847423226, -10.080774619073631], [8.620839623814916, -13.116573654975682], [14.508359276791229, 13.720426753147578], [4.188263790064002, -11.227922947013766], [16.43213549757517, -0.7482393897458062], [8.463467099349561, -10.28666154898164], [-14.590664765392454, 7.512405850588091], [13.518803291309741, -14.339255950978483], [9.13793319122485, -1.7859410526347252], [-0.9933598074793126, -17.200904458717158], [-7.464667928898363, -9.405729236669917], [-0.03825136609191482, 15.996571333365686], [-5.022322703978026, -11.069542356797196], [9.809872137951121, -10.94772821955663], [-4.507997522886509, -16.228093636466127], [-13.12273007268467, -4.660281151003596]]


Now how would we access only 1 element? We use square brackets []!

In [14]:
print vel[0]

[4.655349207021269, -5.407684326983735]


Now how would we access the x_velocity component here? We'd use another set of square brackets!

In [15]:
print vel[0][0]

4.65534920702


Now we need to write a function that will handle the interaction of any two atoms.  

Now lets write a function to calculate the forces on our atoms.

In [ ]:
def forces(coords):
    ''' Compute the forces, the potential energy '''
    
    epsilon = 20.0           # LJ parameter
    sigma = atom_radius*2    # LJ parameter
    
    f = [(0.0,0.0)]*len(coords)
    pe = 0.0
    
    # V(ri-rj) = epsilon*((sigma/r)^12 - 2*(sigma/r)^6)
    # dV/dxi = -12*epsilon*((sigma/r)^14 - (sigma/r)^8)*(xi-xj)/sigma**2
    # F[i][x] = -dV/dxi
    
    fac = epsilon*12.0/sigma**2
    
    for i in range(len(coords)):
        
        for j in range(i+1,len(coords)):
            xi,yi = coords[i].pos()
            xj,yj = coords[j].pos()
            dx,dy = (xi-xj),(yi-yj)
            rsq = (dx**2+dy**2)/sigma**2
        
            # Error if true: print to help debug why
            if rsq == 0.0:
                print "%.8f %.8f %.8f %.8f %d %d" % (xi,yi,xj,yj,i,j)
            
            if rsq < 1.0:
                vij = epsilon*(1.0/rsq**6 - 2.0/rsq**3)
                df = fac*(1.0/rsq**7 - 1.0/rsq**4)
                dfx,dfy = df*dx,df*dy
                fxi,fyi = f[i]
                fxj,fyj = f[j]
                f[i] = fxi+dfx,fyi+dfy
                f[j] = fxj-dfx,fyj-dfy
                pe += vij

    return f, pe

Now to start moving our atoms.

In [ ]:
# The amount of time each iteration moves us forward
dt = 0.01

# Max number of steps we want to take
max_steps = 100000

# Print headers for output
print " "
print "   time         ke            pe             e      "
print " -------    -----------   ------------  ------------"  

# Calculate forces
f, pe = forces(atoms)

for step in range(max_steps):
    
    # For each atom
    for j in range(num_atoms):
    
        # Get the current position, velocity, and force of the atom
        vx,vy = vel[j]    
        Fx,Fy = f[j]  
        vx += Fx*dt*0.5
        vy += Fy*dt*0.5
        x,y = atoms[j].pos()
    
        # Check if moving left or right will put our atom beyond the wall
        if abs(x + dt * vx) >= width/2.0 - atom_radius:
        
            # We have moved too far right or left, so flip vx
            vx = -vx           
    
        # Check if moving up or down will put our atom beyond the wall
        if abs(y + dt * vy) >= height/2.0 - atom_radius:
        
            # We have moved too far up or down, so flip the y_vel
            vy = -vy
        
        # We won't move out of the box, so update the new position
        atoms[j].goto(x + dt*vx, y + dt*vy)
        
        # Also save the new velocities
        vel[j] = [vx, vy]
        
        
    # make the forces at time t+dt
    f, potential_energy = forces(atoms)
            
    # finish update of v to time t+dt
    kinetic_energy = 0.0
    for i in range(num_atoms):
        vx,vy = vel[i]
        Fx,Fy = f[i]
        vx += Fx*dt*0.5
        vy += Fy*dt*0.5
        vel[i] = [vx, vy]
        kinetic_energy += 0.5*(vx*vx+vy*vy)
        
        
    # Tell turtles we are done updating and to redraw
    # Only do this every 10 steps (speeds it up)
    if step % 10 == 0:
        turtle.update()
    
        # Output energies to make sure its working   
        energy = kinetic_energy + potential_energy
        print "%8.2f   %12.5f   %12.5f  %12.5f  " % (step*dt, kinetic_energy, potential_energy, energy) 
    

 
   time         ke            pe             e      
 -------    -----------   ------------  ------------
    0.00     2409.81692        0.00000    2409.81692  
    0.10     2409.81692        0.00000    2409.81692  
    0.20     2409.81692        0.00000    2409.81692  
    0.30     2409.81692        0.00000    2409.81692  
    0.40     2409.81692        0.00000    2409.81692  
    0.50     2409.81692        0.00000    2409.81692  
    0.60     2409.81692        0.00000    2409.81692  
    0.70     2409.81692        0.00000    2409.81692  
    0.80     2409.81692        0.00000    2409.81692  
    0.90     2409.81692        0.00000    2409.81692  
    1.00     2409.81692        0.00000    2409.81692  
    1.10     2409.81692        0.00000    2409.81692  
    1.20     2409.81692        0.00000    2409.81692  
    1.30     2409.81692        0.00000    2409.81692  
    1.40     2409.81692        0.00000    2409.81692  
    1.50     2409.81692        0.00000    2409.81692  
    1.60    

It works!

Always clean up.

In [ ]:
window.bye()